In [32]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import pickle
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, concatenate, Dropout, Input

In [33]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Dropout, InputLayer, Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import model_from_json
import numpy as np
import pandas as pd

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [34]:
from tensorflow.math import exp

def scheduler(epoch, lr):
    if epoch < 20:
        return lr
    else:
        return lr * exp(-0.1)

In [35]:
from tensorflow.keras.callbacks import Callback

class TerminateOnBaseline(Callback):
    """Callback that terminates training when either acc or val_acc reaches a specified baseline
    """
    def __init__(self, monitor='accuracy', baseline=0.9):
        super(TerminateOnBaseline, self).__init__()
        self.monitor = monitor
        self.baseline = baseline

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        acc = logs.get(self.monitor)
        if acc is not None:
            if acc >= self.baseline:
                print('Epoch %d: Reached baseline, terminating training' % (epoch))
                self.model.stop_training = True

In [36]:
parameters_mlp = { 
    'introduction':  [ 0.2, 100, 8],
    'materials':     [ 0.2, 100, 8],
    'conclusion':  [ 0.2, 100, 8]}

# MV

In [37]:
section='conclusion'

with open('dataset/dataset_{}.pkl'.format('features'), 'rb') as fp:
    dataset = pickle.load(fp)

X_features = dataset[section][0]
y_features = dataset[section][2]


columns = list(range(0, 383))
columns = list(map(str, columns))

folder_to_save = 'models_v1'
path_to_save = "/scratch/cinthiasouza/mv-text-summarizer/notebook/{}".format(folder_to_save)

X_embedd = pd.read_csv("dataset/embed_bert_{}_train.csv".format(section))

y_embedd = X_embedd['label']
X_embedd = X_embedd[columns]

In [51]:
len(y_features)

55620

In [48]:
sequence_input = Input(shape=(X_embedd.shape[1],), dtype='int32')

S_1 = Dense(512, activation='tanh')(sequence_input)
S_1 = Dense(512, activation='linear')(S_1)
S_1 = Dense(512, activation="softmax")(S_1)
layer1 = S_1
S_1 = Flatten()(S_1)

sequence_input2 = Input(shape=(X_features.shape[1],), dtype='int32')

S_2 = Dense(512, activation='tanh')(sequence_input2)
S_2 = Dense(512, activation='linear')(S_2)
S_2 = Dense(512, activation='softmax')(S_2)
S_2 = Flatten()(S_2)

S_3 = Dense(512, activation='tanh')(sequence_input2)
S_3 = Dense(512, activation='linear')(S_3)
S_3 = Dense(512, activation="softmax")(S_3)
S_3 = Flatten()(S_3)

v_1 = S_1

v_2_concat = concatenate([v_1, S_2])
v_2 = Dense(512, activation='tanh')(v_2_concat)

v_3_concat = concatenate([v_1, v_2, S_3])
v_3 = Dense(512, activation='tanh')(v_3_concat)

concat_layer = concatenate([v_1, v_2, v_3])

perceptron_1 = Dense(256, activation='relu')(concat_layer)
dropout1 = Dropout(.2)(perceptron_1)
perceptron_2 = Dense(256, activation='relu')(dropout1)
dropout2 = Dropout(.2)(perceptron_2)
perceptron_3 = Dense(512, activation='relu')(dropout2)
dropout3 = Dropout(.3)(perceptron_3)
perceptron_4 = Dense(512, activation='relu')(dropout3)
dropout4 = Dropout(.3)(perceptron_4)
perceptron_5 = Dense(1024, activation='relu')(dropout4)
dropout5 = Dropout(.5)(perceptron_5)
perceptron_6 = Dense(1024, activation='relu')(dropout5)
dropout6 = Dropout(.5)(perceptron_6)
perceptron_7 = Dense(512, activation='relu')(dropout6)
dropout7 = Dropout(.3)(perceptron_7)
perceptron_8 = Dense(256, activation='relu')(dropout7)
dropout8 = Dropout(.3)(perceptron_8)
perceptron_9 = Dense(128, activation='relu')(dropout8)
dropout9 = Dropout(.2)(perceptron_3)

preds = Dense(2, activation='sigmoid')(dropout9)

model = Model(inputs=[sequence_input, sequence_input2], outputs=preds)

my_callbacks = [keras.callbacks.LearningRateScheduler(scheduler),
                TerminateOnBaseline(monitor='val_precision', baseline=0.9)]

model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(
                learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
one_hot_label = to_categorical(y_embedd)
X_train_embedd, X_valid_embedd, y_train_embedd, y_valid_embedd = train_test_split(
    X_embedd, one_hot_label, stratify=one_hot_label, shuffle=True, test_size=0.1)

one_hot_label = to_categorical(y_features)
X_train_features, X_valid_features, y_train_features, y_valid_features = train_test_split(
    X_features, one_hot_label, stratify=one_hot_label, shuffle=True, test_size=0.1)

history= model.fit(
	x=[X_train_embedd, X_train_features], y=y_train_features,
	epochs=5, validation_data=([X_valid_embedd, X_valid_features], y_valid_features),
    callbacks=my_callbacks, shuffle=True, batch_size=4)

Epoch 1/5
12515/12515 [==============================] - 672s 53ms/step - loss: 0.6303 - accuracy: 0.6335 - val_loss: 0.6162 - val_accuracy: 0.6609
Epoch 2/5
12515/12515 [==============================] - 791s 63ms/step - loss: 0.6095 - accuracy: 0.6657 - val_loss: 0.6138 - val_accuracy: 0.6593
Epoch 3/5
12515/12515 [==============================] - 990s 79ms/step - loss: 0.6079 - accuracy: 0.6685 - val_loss: 0.6188 - val_accuracy: 0.6546
Epoch 4/5
 3707/12515 [=======>......................] - ETA: 14:09 - loss: 0.6125 - accuracy: 0.6623

In [42]:
one_hot_label = to_categorical(y_embedd)
X_train_embedd, X_valid_embedd, y_train_embedd, y_valid_embedd = train_test_split(
    X_embedd, one_hot_label, stratify=one_hot_label, shuffle=True, test_size=0.1)

one_hot_label = to_categorical(y_features)
X_train_features, X_valid_features, y_train_features, y_valid_features = train_test_split(
    X_features, one_hot_label, stratify=one_hot_label, shuffle=True, test_size=0.1)

history= model.fit(
	x=[X_train_embedd, X_train_features], y=y_train_features,
	epochs=3, validation_data=([X_valid_embedd, X_valid_features], y_valid_features),
    callbacks=my_callbacks, shuffle=True, batch_size=8)

Epoch 1/3
6258/6258 [==============================] - 347s 55ms/step - loss: 0.6625 - accuracy: 0.6233 - val_loss: 0.6586 - val_accuracy: 0.6341
Epoch 2/3
6258/6258 [==============================] - 320s 51ms/step - loss: 0.6476 - accuracy: 0.6255 - val_loss: 0.6350 - val_accuracy: 0.6458
Epoch 3/3
6258/6258 [==============================] - 319s 51ms/step - loss: 0.6465 - accuracy: 0.6088 - val_loss: 0.6476 - val_accuracy: 0.6471


In [50]:
model_json = model.to_json()
with open('{}/mv_mlp_bert_{}.json'.format(path_to_save, section), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights('{}/mv_mlp_bert_{}.h5'.format(path_to_save, section))
print("Saved model to disk")

Saved model to disk


In [159]:
model.evaluate(x=[X_train_embedd, X_train_features], y=y_train_features)

1535/1535 [==============================] - 6s 4ms/step - loss: 0.5657 - precision_12: 0.6864


[0.565717339515686, 0.6863737106323242]